In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from main import *
from v_p_chapa_cabeca import *
from materiais import *

In [2]:
perfil_escolhido = W_360x58_0
perfil_escolhido.inercias()

aço_escolhido = ASTM_A36

solda_escolhida =E70XX

##### Cálculo da disposição da chapa, dos parafusos e do diâmetro minimo

In [3]:
M = 248000 #kN.mm 

V = 240 #kN

T = 360 #kN

parafuso = ASTM_A325

diametros = diametros_A325

rosca = 1

planos_de_corte =1

S = curva_interacao(M,V,T,perfil_escolhido,parafuso,rosca,planos_de_corte,diametros,gamma_a2)

chapa = S[3]
ver_parafuso = S[4].df

N = len(ver_parafuso)  #Número total de parafusos
n = (ver_parafuso["x (mm)"] == ver_parafuso["x (mm)"].iloc[0]).sum()  #número de parafusos por coluna
n_p_c = N/n  #número de parafusos por camada

S[1]

'1.1/8'

In [4]:
chapa.df

,vértice,x (mm),y (mm)
0,1,0,0
1,2,200,0
2,3,200,458
3,4,0,458
4,5,0,0


##### Cálculo da espessura da chapa de cabeça

In [5]:
rigida = 1

F_r_total = resistencia_total(parafuso,rosca,planos_de_corte,gamma_a2)

F_t_Sd=solicitante_parafuso_tração(M,chapa.B, np.unique(ver_parafuso["y (mm)"]),n_p_c, parafuso , n, S[0]) + T/(n_p_c*n)

exp = exp_placa(aço_escolhido, chapa, espessuras, rigida, ver_parafuso, pol_to_mm(S[1]), F_r_total,F_t_Sd,gamma_a1)

exp = pol_to_mm(exp)

exp

22.224999999999998

##### Cálculo da espessura do filete de solda na alma

In [6]:
espessura_solda_alma = esp_solda_alma(perfil_escolhido,aço_escolhido,solda_escolhida,(perfil_escolhido.h_w),exp,M,V,T,True,gamma_a1,gamma_a2)

math.ceil(espessura_solda_alma) #mm

7

In [7]:
espessura_solda_mesa = esp_solda_mesa(perfil_escolhido,aço_escolhido,solda_escolhida,(perfil_escolhido.b_f),exp,M,V,T,True,gamma_a1,gamma_a2)

math.ceil(espessura_solda_mesa) #mm

8

In [8]:
from pyautocad import Autocad, APoint
from comtypes import npsupport
npsupport.enable()

In [9]:
acad = Autocad(create_if_not_exists=True)
acad.prompt("Hello, Autocad from Python\n")
print(acad.doc.Name)

Hello, Autocad from Python

Drawing5.dwg


In [10]:
p1 = APoint(0, 0)
p2 = APoint(50, 25)
for i in range(5):
    text = acad.model.AddText(u'Hi %s!' % i, p1, 2.5)
    acad.model.AddLine(p1, p2)
    acad.model.AddCircle(p1, 10)
    p1.y += 10

In [11]:
pontos=chapa.df

In [12]:
pontos.shape[0]

5

In [13]:
for i in range(0,pontos.shape[0]-1):
    acad.model.AddLine(APoint(pontos.iat[i,1],pontos.iat[i,2]),APoint(pontos.iat[i+1,1],pontos.iat[i+1,2]))

In [28]:
for obj in acad.iter_objects():
    print(obj.ObjectName)

AcDbText
AcDbLine
AcDbCircle
AcDbText
AcDbLine
AcDbCircle
AcDbText
AcDbLine
AcDbCircle
AcDbText
AcDbLine
AcDbCircle
AcDbText
AcDbLine
AcDbCircle
